In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-05 05:14:54--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-12-05 05:14:54 (10.8 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://dshbigdatahw.s3.us-east-2.amazonaws.com/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
# https://dshactivity3-2021.s3.us-east-2.amazonaws.com/user_data.csv
spark.sparkContext.addFile(url)
review_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [6]:
print(review_df.count())
clean_review_df = review_df.dropna()
print(clean_review_df.count())
cleaner_review_df = clean_review_df.dropDuplicates()
print(cleaner_review_df.count())


904765
904663
904663


In [7]:
cleaner_review_df.printSchema()


root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [9]:
from pyspark.sql.types import IntegerType,BooleanType,DateType

cleaner_review_df = cleaner_review_df.withColumn("review_date",cleaner_review_df.review_date.cast(DateType()))

In [10]:
cleaner_review_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [11]:
review_id_df = cleaner_review_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100LMJJNWFDJK|   12078619|B00B054NVG|     963648561| 2014-10-24|
|R10106OQ9QP86T|   43421823|B004W4TYMA|     461512942| 2015-03-30|
|R101ER44F5J3EK|   49269789|B00AM966CU|     172551627| 2015-02-04|
|R1036AJM4DUM64|   17911630|B006X735PQ|     627151806| 2013-04-03|
|R103CWY3MPPRYB|   31063425|B00DY1F2CS|     554396934| 2015-08-02|
|R104VHJ7WLB4PP|    7146158|B00J0823FI|     647000350| 2015-06-11|
|R105A5ZRV1UHT1|   15499595|B0002H0FZ6|     497097576| 2014-10-18|
|R105RU48J2VL19|   17984606|B005DBF0AG|     990545730| 2015-06-15|
|R106LKOCWLJKP1|   14027720|B00558JGN4|     463070117| 2013-02-13|
|R106NRR8JSGUID|   20032109|B000VTMUQ2|     477030445| 2014-07-05|
|R106V7A3TQ944V|   41138518|B0002KZIV4|     741135212| 2012-12-10|
|R109H6GZEOXX1S|   25494035|B0016MNBAM|     288892337| 2011-09

In [12]:
products_df = cleaner_review_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00B054NVG|Odyssey FZGSPIDDJ...|
|B004W4TYMA|BEHRINGER EUROPOW...|
|B00AM966CU|LEDJump Data Repe...|
|B006X735PQ|Snark SN-2 All In...|
|B00DY1F2CS|NEEWER Adjustable...|
|B00J0823FI|Hercules DG400BB ...|
|B0002H0FZ6|D'Addario Bronze ...|
|B005DBF0AG|Adjustable Artist...|
|B00558JGN4|gemini dj Firstmi...|
|B000VTMUQ2|Dunlop Tortex Sta...|
|B0002KZIV4|Fender Strat Knob...|
|B0016MNBAM|EMUSIC New Versio...|
|B001TYLS98|Black 3.5mm Male ...|
|B0002M6CVC|Ernie Ball slinky...|
|B00F20MIC6|Fretfunk® Acousti...|
|B0002IL3AQ|Django Reinhardt ...|
|B0014SP0GQ|Classic 'C' Flute...|
|B000AJIF4E|Sony MDR7506 Prof...|
|B001LTZZSG|Kala MK Makala Uk...|
|B004OK17QS|TC Electronic Hal...|
+----------+--------------------+
only showing top 20 rows



In [13]:
customers_df = cleaner_review_df.groupby(["customer_id"]).count()
customers_df =customers_df.withColumnRenamed('count' , 'customer_count')
# df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
# df_bostonLegible = df_boston.withColumnRenamed("zn", "Zoning")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10256687|             3|
|   38115097|             1|
|   45891545|             5|
|   35646935|             1|
|    5373021|             1|
|    2533410|            11|
|   12886089|             1|
|   45599268|             1|
|   44267121|             8|
|   42984093|             1|
|   39095180|             1|
|   25232993|             1|
|   11263585|             5|
|   29660587|             3|
|   24652965|             3|
|   52600605|             2|
|   10012354|             2|
|   10208065|             2|
|   43381859|             1|
|   42208849|            11|
+-----------+--------------+
only showing top 20 rows



In [14]:
vine_table_df = cleaner_review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100LMJJNWFDJK|          5|            0|          0|   N|
|R10106OQ9QP86T|          5|            0|          1|   N|
|R101ER44F5J3EK|          5|            0|          0|   N|
|R1036AJM4DUM64|          4|            0|          0|   N|
|R103CWY3MPPRYB|          2|            0|          0|   N|
|R104VHJ7WLB4PP|          5|            0|          0|   N|
|R105A5ZRV1UHT1|          5|            0|          0|   N|
|R105RU48J2VL19|          5|           12|         12|   N|
|R106LKOCWLJKP1|          1|            2|          8|   N|
|R106NRR8JSGUID|          5|            0|          0|   N|
|R106V7A3TQ944V|          5|            0|          0|   N|
|R109H6GZEOXX1S|          5|            2|          2|   N|
|R10ACINEOPG32H|          1|            0|          0|   N|
|R10BNDWVSS8YML|          5|            

Postgres Setup

In [15]:


#  Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cl5tdbfnev78.us-east-2.rds.amazonaws.com:5432/HW2"
config = {"user":"root", 
          "password": "xxxrdexxx", 
          "driver":"org.postgresql.Driver"}


In [16]:
# Write DataFrame to review_id_table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# Write dataframe to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [18]:
# Write dataframe to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [19]:
# Write dataframe to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)